In [ ]:
import numpy as np
import pandas as pd
import csv
import nltk
import re

# read in description txt (5 sentences each)
def read_des_train():
    train_path = './data/descriptions_train/' 
    des_train = []
    for i in range(10000):
        file_name = str(i) + '.txt'
        file_path = train_path + file_name
        des = []
        with open(file_path) as f:
            for line in f.readlines():
                des.append(line.strip('\n'))
        des_train.append(des)
    return des_train


desc_train = read_des_train()

# read in tags
def read_tag_train():
    train_path = './data/tags_train/' 
    tag_train = []
    cat_list = []
    for i in range(10000):
        file_name = str(i) + '.txt'
        file_path = train_path + file_name
        tag = ''
        with open(file_path) as f:
            for line in f.readlines():
                sup_cat, sub_cat = line.strip('\n').split(':')
                tag = tag + ' ' + sub_cat
                cat_list.append(sub_cat)
        tag.strip()
        tag_train.append(tag)
    return tag_train, cat_list

tags_train, category_list = read_tag_train()
category_list = list(set(category_list))

#print tags_train   # empty space in front of each? 
#print category_list
print len(category_list)
print tags_train
    

# bag of words
def process_BoW(desc_data):
    stopwords = nltk.corpus.stopwords.words('english')
    stemmer = nltk.SnowballStemmer("english")
    desc_words_bow = []
    m = 0
    for descs in desc_data:
        temp = ''
        for i in range(len(descs)):
            if m == 20:
                print descs[i]
            word_lower = descs[i].lower()
            if m == 20:
                print word_lower
            word_list = re.findall(r"[a-z]+", word_lower)
            if m == 20:
                print word_list
            word_filter = [stemmer.stem(word) for word in word_list if word not in stopwords]
            if m == 20:
                print word_filter 
            for j in range (len(word_filter)):
                temp = temp + ' '+word_filter[j] 
            if m == 20:
                print temp 
            m = m +1
        desc_words_bow.append(temp)
    return desc_words_bow

#dict_bow = process_BoW(desc_train)
desc_word_bow = process_BoW(desc_train)

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

cv = CountVectorizer(min_df = 5) # frequency threshold
x_bow_train = cv.fit_transform(desc_word_bow)
# print x_bow_train.toarray()
words_extract = np.array(cv.get_feature_names())

from yellowbrick.text import FreqDistVisualizer
from yellowbrick.features import Rank1D

import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, figsize=(10, 14))
visualizer = FreqDistVisualizer(features=cv.get_feature_names())
visualizer.fit(x_bow_train)
visualizer.poof(outpath="freq.png")

print len(words_extract)
print type(words_extract)

transformer = TfidfTransformer()
x_tf_train = transformer.fit_transform(x_bow_train).toarray()

fig, ax = plt.subplots(1, figsize=(10, 14))
visualizer = FreqDistVisualizer(features=words_extract)
visualizer.fit(x_tf_train)
# visualizer.poof(outpath="freq_tfid.png")

# Convert into DataFrames
X_train_tfidf = pd.DataFrame(x_tf_train, columns = words_extract)
X_train_bow = pd.DataFrame(x_bow_train.toarray(), columns = words_extract)
X_train_tfidf.head(20)
# X_train_bow.head(20)

x_tf_train = pd.DataFrame(x_tf_train, columns = words_extract)

cv = CountVectorizer(vocabulary = category_list)
tags_train_c = cv.fit_transform(tags_train).toarray()

def read_des_test():
    train_path = './data/descriptions_test/' 
    des_test = []
    for i in range(2000):
        file_name = str(i) + '.txt'
        file_path = train_path + file_name
        des = []
        with open(file_path) as f:
            for line in f.readlines():
                des.append(line.strip('\n'))
        des_test.append(des)
    return des_test

desc_test = read_des_test()

desc_test_bow = process_BoW(desc_test) # To do: process BOW 


# process test data


cv = CountVectorizer(vocabulary = words_extract)
x_bow_test = cv.fit_transform(desc_test_bow)
transformer = TfidfTransformer()
x_test_tf = transformer.fit_transform(x_bow_test).toarray()

# Convert into DataFrames
x_test_tf = pd.DataFrame(x_test_tf, columns = words_extract)



print len(x_test_tf)

# PCA 
from sklearn.decomposition import PCA

pca_pool5 = PCA(n_components=2048, svd_solver='auto') # 2048 -> change threshold 
pca_pool5.fit(x_tf_train)
bow_all_train_PCA = pca_pool5.transform(x_tf_train)
#bow_all_test_PCA = pca_all_pool5.transform(bow_all_test)

plt.scatter(bow_all_train_PCA[:,0], bow_all_train_PCA[:,1], c=data.target)
plt.show()

pca_test = PCA(n_components=2048, svd_solver='auto') # 2048 -> change threshold 
#pca_test.fit(x_test_tf)
test_PCA = pca_pool5.transform(x_test_tf)


pool5_train_data = pd.read_csv(filepath_or_buffer="./data/features_train/features_resnet1000intermediate_train.csv",header=None
)
pool5_train_data[0] = pool5_train_data[0].apply(lambda x: int(x.split('/')[1].replace('.jpg','')))
pool5_train_sort = pool5_train_data.sort_values(by=[0])
pool5_train_images = pool5_train_sort[0].values
pool5_train = pool5_train_sort[list(range(1,2049))].values

# pool5 test
pool5_test_data = pd.read_csv(filepath_or_buffer="./data/features_test/features_resnet1000intermediate_test.csv", header=None
)
pool5_test_data[0] = pool5_test_data[0].apply(lambda x: int(x.split('/')[1].replace('.jpg','')))
pool5_test_sort = pool5_test_data.sort_values(by=[0])
pool5_test_images = pool5_test_sort[0].values
pool5_test = pool5_test_sort[list(range(1,2049))].values

# from sklearn.cross_decomposition import PLSRegression

# pls_all_pool5 = PLSRegression(n_components=1224)
# pls_all_pool5.fit(bow_all_train_PCA, pool5_train)


# Ramdom Forest

from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

clf = RandomForestRegressor(n_jobs=-1)
clf.fit(bow_all_train_PCA, pool5_train)

#predict_tags = []
"""
for i in range(80):
    print i
    y = tags_train_c[:,i]
    clf = RandomForestRegressor(n_jobs=-1)
    clf.fit(bow_all_train_PCA, pool5_train)
    tag = clf.predict(x_test_tf)
    predict_tags.append(tag)
  """  

from sklearn.neighbors import NearestNeighbors
nbs = NearestNeighbors(n_neighbors=20, metric='cosine').fit(pool5_test)
#np.array([test_PCA[5]])
predict = clf.predict(np.array([test_PCA[5]]))
distance, idx = nbs.kneighbors(predict) #success here

predict_p5 = []
for i in range(2000):
    predict = clf.predict(np.array([test_PCA[i]]))
    #predict_p5.append(predict)
    distance, idx = nbs.kneighbors(predict) #success here
    predict_p5.append(idx[0].tolist())

print predict_p5



predict_tags = np.array(predict_tags)
predict_tags_t = predict_tags.T


def read_tag_test():
    train_path = './data/tags_test/' 
    tag_test = []
    for i in range(2000):
        file_name = str(i) + '.txt'
        file_path = train_path + file_name
        tag = ''
        with open(file_path) as f:
            for line in f.readlines():
                sup_cat, sub_cat = line.strip('\n').split(':')
                tag = tag + ' ' + sub_cat
        tag.strip()
        tag_test.append(tag)
    return tag_test #, cat_list

tags_test = read_tag_test()

cv = CountVectorizer(vocabulary = category_list)
tags_test_c = cv.fit_transform(tags_test)


# knn
from sklearn.neighbors import NearestNeighbors as KNN

label_test = []
for i in range(2000):
    label_test.append(i)
knn = KNN(n_neighbors = 20, n_jobs=-1)
knn = knn.fit(tags_test_c, label_test)
prediction = knn.kneighbors(predict_tags_t, return_distance = False)

img_list = []
for row in (predict_p5): # pool5 pca predictions 
    img_num = ''
    for val in row:
        img_num = img_num + ' ' + (str(val) + ".jpg")
    img_list.append(img_num)
    
images = pd.DataFrame(img_list, columns = ["Top_20_Image_IDs"])

idx = []
for i in range(2000):
    idx.append(str(i)+ ".txt")
idx = pd.DataFrame(idx, columns = ["Descritpion_ID"])


result = pd.concat([idx, images], axis=1)
result.to_csv("test_p5PCA_result.csv", index = False)